In [1]:
# Lasciate questa cella così ogni volta che aggiornate un file python, 
# anche gli import nel notebook vengono aggiornati

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from Utils import *
from constants import *

from Environment import *
from Non_Stationary_Environment import *

from Social_influence import *
from Network import Network

from Simulation import *

from Ecommerce import *
from Ecommerce3 import *
from Ecommerce4 import *
from Ecommerce5 import *
from Ecommerce6 import *
from step7.Ecommerce7 import*

## Step 2

In [3]:
simulate_step2()

optimal allocation is: [60. 60. 20. 60.  0.] with a reward of: 457
estimated_opt_allocation is: [60. 60. 20. 60.  0.] with a reward of: 457


## Step 3

In [ ]:
gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts = simulate_step3()
plot_regrets(gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts, ["GPTS", "GPUCB"])

## Step 4

In [11]:
gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts = simulate_step4()
plot_regrets(gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts, ["GPTS", "GPUCB"])

## Step 5

In [ ]:
gpucb_rewards_per_experiment, gpts_rewards_per_experiment, opts = simulate_step5()
plot_regrets(gpts_rewards_per_experiment, gpucb_rewards_per_experiment, opts, ["GPTS", "GPUCB"])

## Step 6

In [ ]:
swucb_rewards_per_experiment, cducb_rewards_per_experiment, opts = simulate_step6()
plot_regrets(swucb_rewards_per_experiment, cducb_rewards_per_experiment, opts, ["SWUCB", "CDUCB"])

## Step 7

In [3]:
B_cap = 100
budgets = np.arange(start=0, stop=B_cap + 1, step=5)

In [4]:
env, observations_probabilities, click_probabilities, product_prices,\
    users_reservation_prices, users_poisson_parameters = generate_new_environment()

In [5]:
nodes_activation_probabilities, num_sold_items = estimate_nodes_activation_probabilities(
        click_probabilities,
        users_reservation_prices,
        users_poisson_parameters,
        product_prices,
        observations_probabilities
    )

In [24]:
aggr_exp_num_clicks = env.estimate_num_of_clicks(budgets / B_cap)

In [25]:
ecomm = Ecommerce(B_cap, budgets, product_prices)
ecomm.solve_optimization_problem(num_sold_items, aggr_exp_num_clicks, nodes_activation_probabilities)

(array([20, 45, 20, 15,  0]), 391.05226280394925)

In [27]:

num_of_items_sold_for_each_product = np.sum(
    num_sold_items, axis=0)  # shape = 1x5


total_margin_for_each_product = np.multiply(
    num_of_items_sold_for_each_product, product_prices)  # shape = 1x5

value_per_click = np.dot(
    nodes_activation_probabilities, total_margin_for_each_product.T)

value_per_click = np.repeat(value_per_click, aggr_exp_num_clicks.shape[-1], axis = -1).reshape(aggr_exp_num_clicks.shape)

exp_reward = np.multiply(aggr_exp_num_clicks, value_per_click)

values = exp_reward.ravel()
weights = np.tile(budgets, reps = NUM_OF_PRODUCTS)


In [6]:
dis_exp_num_clicks = env.estimate_disaggregated_num_clicks(budgets/B_cap)

In [7]:
tot_margin_per_product = np.multiply(num_sold_items, product_prices) # NUM_OF_USER_CLASSES x NUM_OF_PRODUCTS

value_per_click = np.dot(tot_margin_per_product, nodes_activation_probabilities.T) # NUM_OF_USER_CLASSES x NUM_OF_PRODUCT

value_per_click = np.repeat(value_per_click, dis_exp_num_clicks.shape[-1], axis = -1).reshape(dis_exp_num_clicks.shape)

exp_reward = np.multiply(dis_exp_num_clicks, value_per_click).reshape((NUM_OF_USERS_CLASSES*NUM_OF_PRODUCTS, budgets.shape[0]))

values = exp_reward.ravel()
weights = np.tile(budgets, reps = NUM_OF_USERS_CLASSES* NUM_OF_PRODUCTS)

In [28]:
from functools import lru_cache

class knapsack:
    def __init__(self, cost, reward):
        assert(cost.shape == reward.shape)
        self.cost = cost
        self.reward = reward

    @lru_cache(maxsize=4096)
    def solve(self, cap, i=0):
        if cap < 0:
            return -sum(self.reward), []
        if i == len(self.cost):
            return 0, []
        res1 = self.solve(cap, i + 1)
        res2 = self.solve(cap - self.cost[i], i + 1)
        res2 = (res2[0] + self.reward[i], [i] + res2[1])
        return res1 if res1[0] >= res2[0] else res2


In [29]:
reward, optimal_allocation = knapsack(weights, values).solve(B_cap)

(943.5829791152217, [1, 2, 3, 4, 5, 65, 66])

In [22]:
reward

155.6871144159362

In [19]:
rows, cols = np.unravel_index(optimal_allocation, shape=exp_reward.shape)

In [20]:
indices = list(zip(rows, cols))

In [21]:
indices

[(0, 4), (5, 1), (5, 2), (5, 3), (5, 4), (8, 1), (8, 4), (13, 1)]

In [23]:
for idx in indices:
    print(exp_reward[idx])

25.220895077178213
11.97838958083163
26.03085867160024
27.7188915117803
27.7188915117803
6.9488963651783235
23.814735471392112
6.2555562261950906


In [140]:
exp_reward[indices].shape

(9, 2, 21)